<a href="https://colab.research.google.com/github/MohanZhu0623/Sentiment_Analysis/blob/main/Combined_TM(neutral).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install contextualized-topic-models

In [2]:
%%capture
!pip install pyldavis

In [3]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

In [4]:
!head -n 2 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and


In [6]:
import pandas as pd
df = pd.read_excel('/content/kickstarter_data_sampled_sentiment&WC.xlsx')
text_data = df[df['sentiment'] == 0]['blurb'].tolist()

In [7]:
# Print the number of rows
print(f'Number of rows: {len(text_data)}')

Number of rows: 7633


In [8]:
!pip install --upgrade scipy gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 4.2.0
    Uninstalling gensim-4.2.0:
      Successfully uninstalled gensim-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contextualized-topic-models 2.5.0 requires gensim==4.2.0, but you have gensim 4.3.3 which is incompatible.


In [9]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [10]:
from nltk.corpus import stopwords as stop_words

nltk.download('stopwords')

documents = [line.strip() for line in text_data]

stopwords = list(stop_words.words("english"))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation

tp = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [12]:
tp.vocab[:10]

array(['able', 'abstract', 'academy', 'accepted', 'access', 'accessible',
       'accessories', 'accessory', 'acclaimed', 'accompanying'],
      dtype=object)

In [13]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=10)
ctm.fit(training_dataset)   # run the model

0it [00:00, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch: [10/10]	 Seen Samples: [76160/76280]	Train Loss: 65.49810159506919	Time: 0:00:02.065990: : 10it [00:26,  2.64s/it]
100%|██████████| 120/120 [00:01<00:00, 65.93it/s]


In [14]:
# Get the top 10 words for each topic
topic_lists = ctm.get_topic_lists(10)

# Output the top 10 words for each topic line by line
for i, topic in enumerate(topic_lists):
    print(f'Topic {i + 1}: {", ".join(topic)}')

Topic 1: technology, platform, data, users, app, providing, building, software, social, used
Topic 2: need, money, want, start, raise, business, funds, like, looking, create
Topic 3: album, record, recording, songs, music, first, cd, studio, band, ep
Topic 4: photographic, book, history, paintings, photos, images, photographs, illustrated, photo, sculptures
Topic 5: album, record, help, first, release, length, recording, full, need, fund
Topic 6: print, want, like, would, prints, make, coffee, hand, shirts, designs
Topic 7: dance, music, new, show, festival, performance, live, theater, theatre, th
Topic 8: film, comedy, theatre, documentary, short, festival, musical, feature, performance, world
Topic 9: book, art, photo, photography, paintings, images, pages, photographs, drawings, photographer
Topic 10: comic, series, stories, inspired, anthology, fantasy, enamel, graphic, pins, hard
Topic 11: art, artists, project, create, space, magazine, artist, work, community, exhibition
Topic 12

In [15]:
topic_lists = ctm.get_topic_lists(5)
print(topic_lists)

[['technology', 'platform', 'data', 'users', 'app'], ['need', 'money', 'want', 'start', 'raise'], ['album', 'record', 'recording', 'songs', 'music'], ['photographic', 'book', 'history', 'paintings', 'photos'], ['album', 'record', 'help', 'first', 'release'], ['print', 'want', 'like', 'would', 'prints'], ['dance', 'music', 'new', 'show', 'festival'], ['film', 'comedy', 'theatre', 'documentary', 'short'], ['book', 'art', 'photo', 'photography', 'paintings'], ['comic', 'series', 'stories', 'inspired', 'anthology'], ['art', 'artists', 'project', 'create', 'space'], ['help', 'fund', 'need', 'raise', 'get'], ['playing', 'cards', 'deck', 'enamel', 'limited'], ['music', 'new', 'dance', 'video', 'performance'], ['film', 'series', 'short', 'feature', 'story'], ['service', 'food', 'social', 'app', 'online'], ['art', 'artists', 'public', 'project', 'installation'], ['games', 'real', 'mobile', 'compatible', 'game'], ['stories', 'novel', 'series', 'anthology', 'short'], ['game', 'android', 'rpg', 'g

In [16]:
lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

100%|██████████| 120/120 [00:05<00:00, 21.98it/s]


In [17]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

  0%|          | 0/120 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
 98%|█████████▊| 118/120 [00:01<00:00, 77.97it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 120/120 [00:01<00:00, 68.49it/s]
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
